In [1]:
pip install --upgrade transformers accelerate pydub -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 16.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from pydub import AudioSegment
import os

# Load the audio file
audio = AudioSegment.from_wav("/content/audio.wav")

# Set parameters for slicing
segment_duration_ms = 60 * 1000  # 60 seconds in milliseconds
overlap_duration_ms = 3 * 990   # Si le prof parle vite vaut mieux le mettre bas (sinon ya plein de répétitions), si le prof parle très lentement, ça peut être mit + haut

# Create the output directory
output_dir = "/content/trim/"
os.makedirs(output_dir, exist_ok=True)

# Calculate the number of chunks required
total_duration_ms = len(audio)
chunks = []
start_time = 0

# Split the audio into segments with overlap
i = 0
while start_time < total_duration_ms:
    end_time = start_time + segment_duration_ms
    chunk = audio[start_time:end_time]
    chunk.export(f"{output_dir}part_{i}.wav", format="wav")
    chunks.append(f"part_{i}.wav")

    # Update the start time for the next chunk (with overlap)
    start_time += (segment_duration_ms - overlap_duration_ms)
    i += 1

chunks

In [5]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import os
import re

# Set device and precision
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# Load Whisper model and processor
model_id = "openai/whisper-large-v3"
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True
)
model.to(device)
processor = AutoProcessor.from_pretrained(model_id)

# Initialize the ASR pipeline
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    chunk_length_s=30,
    batch_size=18,  # Adjust batch size according to your hardware
    torch_dtype=torch_dtype,
    device=device,
)

# Directory containing .wav files
audio_dir = "/content/drive/MyDrive/trim"
transcription_file = "/content/transcription.txt"

# Clear existing transcription file (if needed)
open(transcription_file, 'w').close()

# Helper function to extract the numeric part of filenames
def get_numeric_part(filename):
    match = re.search(r'part_(\d+)\.wav', filename)
    return int(match.group(1)) if match else None

# Sort files numerically based on the part number
wav_files = sorted(
    [f for f in os.listdir(audio_dir) if f.endswith(".wav")],
    key=get_numeric_part
)

# Iterate over sorted .wav files
for filename in wav_files:
    file_path = os.path.join(audio_dir, filename)

    # Load and process the audio file
    print(f"Processing {filename}...")
    result = pipe(file_path)

    # Extract transcription text
    transcription = result["text"]

    # Save the transcription to the text file
    with open(transcription_file, 'a') as f:
        f.write(transcription + "\n\n")

print(f"Transcriptions saved to {transcription_file}.")

Processing part_0.wav...


/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Processing part_1.wav...


/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Processing part_2.wav...


...

Processing part_121.wav...


/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Processing part_122.wav...


/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Transcriptions saved to /content/transcription.txt.
